In [1]:
import os
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from base64 import b64decode

script_dir = os.path.abspath("")
mymodule_dir = os.path.join(script_dir, "..", "scripts")
sys.path.append(mymodule_dir)
import cleaning

jobs = cleaning.prep(pd.read_pickle("../data/jobs.pkl"))
jobs.head()

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,...,Contact,Vollzeit,Teilzeit,Zeit flexibel,Remote,Job Kategorie,Junior,Gehalt_min_yearly,Gehalt_max_yearly,Gehalt_durchschnitt_yearly
0,(Junior) Data Scientist (m/w/d),KMH GmbH,Köln,4032,5726,57650956,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Teilzeit, Home Office möglich",Wir suchen in Voll- oder Teilzeit\nWir suchen ...,...,Wir freuen uns über deine Bewerbung!\n\n KMH G...,True,True,True,True,Data Scientist,True,48384,68712,58548.0
1,Junior Data Scientist (m/w/d),TUI Cruises GmbH,Hamburg,3548,5081,57605321,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",TUI Cruises ist einer der führenden Kreuzfahrt...,...,TUI Cruises GmbH \nHeidenkampsweg 58\n20097 Ha...,True,False,False,True,Data Scientist,True,42576,60972,51774.0
2,Junior Data Scientist (m/f/x),TUI Cruises GmbH,Hamburg,3548,5081,57605323,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",TUI Cruises is one of the leading cruise provi...,...,TUI Cruises GmbH \nHeidenkampsweg 58\n20097 Ha...,True,False,False,True,Data Scientist,True,42576,60972,51774.0
3,(Junior) Data Scientist / Data Analyst (w/m/d),1&1,"Karlsruhe, Montabaur, Maintal",3871,6129,57608864,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Mit rund 15 Mio. Kundenverträgen und 3.200 Mit...,...,,True,False,False,True,Data Scientist,True,46452,73548,60000.0
4,Junior Data Scientist (m/w/d) – Risikomodelle,Sparkassen Rating und Risikosysteme GmbH,Berlin,3710,5161,57181289,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Die Sparkassen Rating und Risikosysteme (SR) i...,...,Herr Emanuel Reitzenstein Tel. +49 30 20672-0 ...,True,False,False,True,Data Scientist,True,44520,61932,53226.0


In [2]:
def file_as_pdf(df_, suffix):
	options = FirefoxOptions()
	options.add_argument("--headless")

	with webdriver.Firefox(options=options) as driver:
		wait = WebDriverWait(driver, 20)

		first = True
		for i, row in df_.iterrows():
			link = row["Link"]
			JobID = row["JobID"]
			driver.get(link)
			if first:
				# Überprüfen, ob es einen Cookie-Banner gibt
				wait.until(
					lambda driver: len(
						driver.find_elements(
							By.CSS_SELECTOR,
							"#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button",
						)
					)
					> 0
				)
				# Klicke ersten Teil des Cookie Banners
				driver.find_element(
					By.CSS_SELECTOR,
					"#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button",
				).click()

				# Klicke zweiten Teil des Cookie Banners
				wait.until(
					lambda driver: len(
						driver.find_elements(
							By.CSS_SELECTOR,
							".secondary-button.ccmgt_reject_button",
						)
					)
					> 0
				)
				driver.find_element(
					By.CSS_SELECTOR, ".secondary-button.ccmgt_reject_button"
				).click()
				first = False

			if len(driver.find_elements(By.CLASS_NAME, "js-app-ld-ListingOfflineBlock")) == 0:
				pdf = b64decode(driver.print_page())

				with open(f"../output/anzeigen/{JobID}_{suffix}.pdf", 'wb') as f:
					f.write(pdf)
			else:
				df_.drop(i)

In [3]:
jobs[jobs["Ort"] == "Braunschweig"].groupby("Unternehmen")["Titel"].count().sort_values(ascending=False).head(20)

Unternehmen
NEW YORKER                4
KOSATEC Computer GmbH     2
AUEL EDV-Beratung GmbH    1
Ameno GmbH                1
ZEISS                     1
Name: Titel, dtype: int64